In [1]:
# Encode:

In [2]:
# Configuration:

import configparser

# Create ConfigParser object
config = configparser.ConfigParser()
config.read("config.ini", encoding="utf-8")  # Ensure UTF-8 encoding

target_variable = config.get("0_ALL", "target_variable")

df_4_eda_pkl = config.get("4_EDA", "df_4_eda_pkl")

drop_columns = config.get("5_ENCODE", "drop_columns").replace("\n", "").split(", ")
utvecklingsstadium_min = config.getint("5_ENCODE", "utvecklingsstadium_min")
utvecklingsstadium_max = config.getint("5_ENCODE", "utvecklingsstadium_max")
ohe_variables = config.get("5_ENCODE", "ohe_variables").replace("\n", "").split(", ")
tae_variables = config.get("5_ENCODE", "tae_variables").replace("\n", "").split(", ")
num_variables = config.get("5_ENCODE", "num_variables").replace("\n", "").split(", ")

df_5_final_pkl = config.get("5_ENCODE", "df_5_final_pkl")
df_5_final_csv = config.get("5_ENCODE", "df_5_final_csv")
df_5_final_chk_csv = config.get("5_ENCODE", "df_5_final_chk_csv")

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
# unpickle - pandas dataframe
df = pd.read_pickle(df_4_eda_pkl)

print(df.shape)

(518, 35)


In [5]:
df

,delomrade,lan,latitud,longitud,groda,sort,jordart,skordear,graderingsdatum,graderingstyp,...,BT_7d_sum,G0_7d_sum,G3_7d_sum,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum,g_year,g_month,g_week
0,SV Skåne,Skåne län,6135000,397000,Höstvete,Elixer,Unknown,2017,2017-04-17,Veckovis,...,0.0,56.0,4.0,0.0,0.0,0.0,0.0,2017,4,16
1,SV Skåne,Skåne län,6135000,397000,Höstvete,Mariboss,Unknown,2017,2017-04-17,Veckovis,...,0.0,56.0,4.0,0.0,0.0,0.0,0.0,2017,4,16
2,SV Skåne,Skåne län,6135000,397000,Höstvete,Praktik,Unknown,2017,2017-04-17,Veckovis,...,0.0,56.0,4.0,0.0,0.0,0.0,0.0,2017,4,16
3,SV Skåne,Skåne län,6135000,397000,Höstvete,Torp,Unknown,2017,2017-04-17,Veckovis,...,0.0,56.0,4.0,0.0,0.0,0.0,0.0,2017,4,16
4,SV Skåne,Skåne län,6135000,397000,Höstvete,Elixer,Unknown,2017,2017-04-24,Veckovis,...,0.0,60.0,4.0,0.0,0.0,0.0,0.0,2017,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,Unknown,2024,2024-04-15,Veckovis,...,100.0,54.0,3.0,0.0,0.0,0.0,0.0,2024,4,16
516,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,Unknown,2024,2024-04-22,Veckovis,...,0.0,60.0,4.0,0.0,0.0,0.0,0.0,2024,4,17
517,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,Unknown,2024,2024-04-26,Veckovis,...,300.0,63.0,4.0,0.0,0.0,0.0,0.0,2024,4,17
518,SV Skåne,Skåne län,6140000,382000,Höstvete,Kask,Unknown,2024,2024-05-03,Veckovis,...,100.0,99.0,17.0,2.0,0.0,0.0,0.0,2024,5,18


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 518 entries, 0 to 519
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   delomrade           518 non-null    object        
 1   lan                 518 non-null    object        
 2   latitud             518 non-null    object        
 3   longitud            518 non-null    object        
 4   groda               518 non-null    object        
 5   sort                518 non-null    object        
 6   jordart             518 non-null    object        
 7   skordear            518 non-null    int64         
 8   graderingsdatum     518 non-null    datetime64[ns]
 9   graderingstyp       518 non-null    object        
 10  utvecklingsstadium  518 non-null    int64         
 11  Bladfläcksvampar    518 non-null    float64       
 12  Sädesbladlus        420 non-null    float64       
 13  WGS84N              518 non-null    float64       
 14 

In [7]:
# Drop unwanted columns:
df.drop(drop_columns, axis=1, inplace=True)
df.shape

(518, 30)

In [8]:
# Drop unwanted rows:
df = df[(df['utvecklingsstadium'] > utvecklingsstadium_min) & (df['utvecklingsstadium'] < utvecklingsstadium_max)]
df.shape

(287, 30)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287 entries, 8 to 519
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   latitud             287 non-null    object 
 1   longitud            287 non-null    object 
 2   groda               287 non-null    object 
 3   sort                287 non-null    object 
 4   jordart             287 non-null    object 
 5   utvecklingsstadium  287 non-null    int64  
 6   Bladfläcksvampar    287 non-null    float64
 7   Sädesbladlus        231 non-null    float64
 8   WGS84N              287 non-null    float64
 9   WGS84E              287 non-null    float64
 10  TM_7d_avg           287 non-null    float64
 11  TX_7d_avg           287 non-null    float64
 12  TN_7d_avg           287 non-null    float64
 13  UM_7d_avg           287 non-null    float64
 14  UX_7d_avg           287 non-null    float64
 15  UN_7d_avg           287 non-null    float64
 16  FM2_7d_avg   

In [10]:
df = df.reset_index(drop=True)

# Categorical Variables:

In [11]:
print(df.select_dtypes(include=['object']).columns.tolist())

['latitud', 'longitud', 'groda', 'sort', 'jordart']


In [12]:
# Check the distribution of categorical variables:
print(df.select_dtypes(include=['object']).nunique())

latitud      5
longitud     9
groda        1
sort        14
jordart      3
dtype: int64


In [13]:
print(df['groda'].value_counts())

groda
Höstvete    287
Name: count, dtype: int64


In [14]:
print(df['sort'].value_counts())

sort
Torp                          55
Brons                         46
Praktik                       41
KWS Kerrin                    26
Mariboss                      25
Nordh                         17
Linus                         16
Bright                        15
Hereford                      13
Kask                           9
Pondus                         8
Elixer                         8
Julius                         5
Sortblandning Absolut 2024     3
Name: count, dtype: int64


In [15]:
print(df['jordart'].value_counts())

jordart
Unknown                       216
Lättlera (15-25 % ler)         55
Leriga jordar (5-15 % ler)     16
Name: count, dtype: int64


In [16]:
# One-Hot Encoding:

ohe_var = ohe_variables
#ohe = OneHotEncoder(categories = 'auto')   # drop='none', dtype=np.int32, handle_unknown='ignore', sparse_output=False
ohe = OneHotEncoder(categories = 'auto', drop=None, sparse_output=False)   # dtype=np.int32, handle_unknown='ignore'
ohe_data = ohe.fit_transform(df[ohe_var])
#ohe_df = pd.DataFrame(data = ohe_data.toarray(), columns = ohe.get_feature_names_out(ohe_var))
ohe_df = pd.DataFrame(data = ohe_data, columns = ohe.get_feature_names_out(ohe_var), index=df.index)

ohe_df

,groda_Höstvete,jordart_Leriga jordar (5-15 % ler),jordart_Lättlera (15-25 % ler),jordart_Unknown
0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0
...,...,...,...,...
282,1.0,0.0,0.0,1.0
283,1.0,0.0,0.0,1.0
284,1.0,0.0,0.0,1.0
285,1.0,0.0,0.0,1.0


In [17]:
#ohe_df.rename(columns={
#    'jordart_Lättlera (15-25 % ler)': 'jordart_Lättlera',
#    'jordart_Leriga jordar (5-15 % ler)': 'jordart_Leriga',
#    'jordart_Mellanlera (25-40 % ler)': 'jordart_Mellanlera',
#    'jordart_Mulljord': 'jordart_Mulljord',
#    'jordart_Sandjord (<5 % ler)': 'jordart_Sandjord',    
#    'jordart_Styv lera (>40 % ler)': 'jordart_Styv_lera',
#    'jordart_Unknown': 'jordart_Unknown',
#    'RGT Reform': 'RGT_Reform',
#    'RGT Koi': 'RGT_Koi'
#}, inplace=True)

#ohe_df

In [18]:
ohe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   groda_Höstvete                      287 non-null    float64
 1   jordart_Leriga jordar (5-15 % ler)  287 non-null    float64
 2   jordart_Lättlera (15-25 % ler)      287 non-null    float64
 3   jordart_Unknown                     287 non-null    float64
dtypes: float64(4)
memory usage: 9.1 KB


In [19]:
from category_encoders import TargetEncoder

tae_var = tae_variables
target = target_variable  # Target variable
tae = TargetEncoder(cols=tae_var)
tae_data = tae.fit_transform(df[tae_var], df[target])
tae_df = pd.DataFrame(data = tae_data, columns = tae_var, index=df.index)

tae_df

,sort
0,9.480673
1,7.425422
2,9.163897
3,9.655908
4,9.480673
...,...
282,10.393924
283,10.393924
284,10.393924
285,10.393924


# Numerical Variables:

In [20]:
print(df.select_dtypes(include=['number']).columns.tolist())

['utvecklingsstadium', 'Bladfläcksvampar', 'Sädesbladlus', 'WGS84N', 'WGS84E', 'TM_7d_avg', 'TX_7d_avg', 'TN_7d_avg', 'UM_7d_avg', 'UX_7d_avg', 'UN_7d_avg', 'FM2_7d_avg', 'CLDM_7d_avg', 'Q0_7d_avg', 'RR_7d_sum', 'BT_7d_sum', 'G0_7d_sum', 'G3_7d_sum', 'G5_7d_sum', 'G7_7d_sum', 'G8_7d_sum', 'G10_7d_sum', 'g_year', 'g_month', 'g_week']


In [21]:
# Remove target variable from num_variables (if it exists)
if target_variable in num_variables:
    num_variables.remove(target_variable)

num_var=num_variables

In [22]:
# Standard Scaler:

standard_scaler = StandardScaler()
sts_data = standard_scaler.fit_transform(df[num_var])
sts_df = pd.DataFrame(data = sts_data, columns = num_var, index=df.index)

sts_df

,utvecklingsstadium,latitud,longitud,WGS84N,WGS84E,TM_7d_avg,TX_7d_avg,TN_7d_avg,UM_7d_avg,UX_7d_avg,...,CLDM_7d_avg,Q0_7d_avg,RR_7d_sum,BT_7d_sum,G0_7d_sum,G3_7d_sum,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum
0,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,-1.901233,-1.255838,-2.538842,-1.001427,-0.176736,...,-1.310332,0.119607,2.397590,-0.829110,-1.292743,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066
1,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,-1.901233,-1.255838,-2.538842,-1.001427,-0.176736,...,-1.310332,0.119607,2.397590,-0.829110,-1.292743,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066
2,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,-1.901233,-1.255838,-2.538842,-1.001427,-0.176736,...,-1.310332,0.119607,2.397590,-0.829110,-1.292743,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066
3,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,-1.901233,-1.255838,-2.538842,-1.001427,-0.176736,...,-1.310332,0.119607,2.397590,-0.829110,-1.292743,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066
4,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,-1.626827,-1.690050,-1.131659,0.167317,-0.478660,...,0.509802,-1.072808,-0.521940,-0.829110,-1.189486,-1.087375,-0.973473,-0.824955,-0.779757,-0.658066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,0.243093,1.557844,-1.765834,-2.842171e-14,3.552714e-15,-0.342717,-0.487994,-0.332914,1.462792,1.151729,...,0.781226,-0.381553,0.130296,2.534206,0.739208,0.494031,0.224502,-0.086721,-0.173382,-0.658066
283,1.183779,1.557844,-1.765834,-2.842171e-14,3.552714e-15,-0.054991,-0.407039,0.136147,1.312122,1.514038,...,0.350141,0.288101,-0.397704,1.693377,1.056354,0.651422,0.292568,-0.086721,-0.173382,-0.658066
284,1.654122,1.557844,-1.765834,-2.842171e-14,3.552714e-15,0.824172,0.623295,0.953654,1.012192,0.849805,...,-0.240604,0.834625,-0.863587,0.011719,1.650082,1.303472,0.946009,0.440588,0.167704,-0.303556
285,-1.011156,1.557844,-1.765834,-2.842171e-14,3.552714e-15,0.043582,-0.107752,0.144188,0.854482,0.970575,...,-0.384299,-0.331869,0.844648,0.011719,-1.148921,-1.019922,-0.946246,-0.824955,-0.779757,-0.658066


In [23]:
sts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   utvecklingsstadium  287 non-null    float64
 1   latitud             287 non-null    float64
 2   longitud            287 non-null    float64
 3   WGS84N              287 non-null    float64
 4   WGS84E              287 non-null    float64
 5   TM_7d_avg           287 non-null    float64
 6   TX_7d_avg           287 non-null    float64
 7   TN_7d_avg           287 non-null    float64
 8   UM_7d_avg           287 non-null    float64
 9   UX_7d_avg           287 non-null    float64
 10  UN_7d_avg           287 non-null    float64
 11  FM2_7d_avg          287 non-null    float64
 12  CLDM_7d_avg         287 non-null    float64
 13  Q0_7d_avg           287 non-null    float64
 14  RR_7d_sum           287 non-null    float64
 15  BT_7d_sum           287 non-null    float64
 16  G0_7d_su

In [24]:
# MinMax Scaler:

minmax_scaler = MinMaxScaler()
mms_data = minmax_scaler.fit_transform(df[num_var])
mms_df = pd.DataFrame(data=mms_data, columns=num_var, index=df.index)

mms_df

,utvecklingsstadium,latitud,longitud,WGS84N,WGS84E,TM_7d_avg,TX_7d_avg,TN_7d_avg,UM_7d_avg,UX_7d_avg,...,CLDM_7d_avg,Q0_7d_avg,RR_7d_sum,BT_7d_sum,G0_7d_sum,G3_7d_sum,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum
0,0.000000,0.0,0.555556,0.0,0.0,0.132625,0.301126,0.000000,0.401175,0.630952,...,0.460317,0.285926,0.486111,0.00,0.026364,0.007220,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,0.555556,0.0,0.0,0.132625,0.301126,0.000000,0.401175,0.630952,...,0.460317,0.285926,0.486111,0.00,0.026364,0.007220,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.0,0.555556,0.0,0.0,0.132625,0.301126,0.000000,0.401175,0.630952,...,0.460317,0.285926,0.486111,0.00,0.026364,0.007220,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.555556,0.0,0.0,0.132625,0.301126,0.000000,0.401175,0.630952,...,0.460317,0.285926,0.486111,0.00,0.026364,0.007220,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0,0.555556,0.0,0.0,0.190263,0.218105,0.275446,0.622863,0.571429,...,0.822222,0.080951,0.050926,0.00,0.051818,0.014440,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,0.421053,1.0,0.000000,0.0,0.0,0.459989,0.447936,0.431794,0.868590,0.892857,...,0.876190,0.199777,0.148148,1.00,0.527273,0.395307,0.276730,0.171779,0.148148,0.000000
283,0.736842,1.0,0.000000,0.0,0.0,0.520425,0.463415,0.523610,0.840011,0.964286,...,0.790476,0.314890,0.069444,0.75,0.605455,0.433213,0.292453,0.171779,0.148148,0.000000
284,0.894737,1.0,0.000000,0.0,0.0,0.705092,0.660413,0.683631,0.783120,0.833333,...,0.673016,0.408838,0.000000,0.25,0.751818,0.590253,0.443396,0.294479,0.231481,0.097561
285,0.000000,1.0,0.000000,0.0,0.0,0.541130,0.520638,0.525184,0.753205,0.857143,...,0.644444,0.208318,0.254630,0.25,0.061818,0.030686,0.006289,0.000000,0.000000,0.000000


In [25]:
mms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   utvecklingsstadium  287 non-null    float64
 1   latitud             287 non-null    float64
 2   longitud            287 non-null    float64
 3   WGS84N              287 non-null    float64
 4   WGS84E              287 non-null    float64
 5   TM_7d_avg           287 non-null    float64
 6   TX_7d_avg           287 non-null    float64
 7   TN_7d_avg           287 non-null    float64
 8   UM_7d_avg           287 non-null    float64
 9   UX_7d_avg           287 non-null    float64
 10  UN_7d_avg           287 non-null    float64
 11  FM2_7d_avg          287 non-null    float64
 12  CLDM_7d_avg         287 non-null    float64
 13  Q0_7d_avg           287 non-null    float64
 14  RR_7d_sum           287 non-null    float64
 15  BT_7d_sum           287 non-null    float64
 16  G0_7d_su

In [26]:
# Cyclic Encoding:

# cye_df = pd.DataFrame()
# cye_df['year'] = df['year']
# cye_df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
# cye_df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
# cye_df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
# cye_df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)

cye_df = pd.DataFrame({
    'g_year': df['g_year'],
    'g_month_sin': np.sin(2 * np.pi * df['g_month'] / 12),
    'g_month_cos': np.cos(2 * np.pi * df['g_month'] / 12),
    'g_week_sin': np.sin(2 * np.pi * df['g_week'] / 52),
    'g_week_cos': np.cos(2 * np.pi * df['g_week'] / 52)
},index=df.index)   

cye_df

,g_year,g_month_sin,g_month_cos,g_week_sin,g_week_cos
0,2017,5.000000e-01,-0.866025,0.822984,-0.568065
1,2017,5.000000e-01,-0.866025,0.822984,-0.568065
2,2017,5.000000e-01,-0.866025,0.822984,-0.568065
3,2017,5.000000e-01,-0.866025,0.822984,-0.568065
4,2017,5.000000e-01,-0.866025,0.748511,-0.663123
...,...,...,...,...,...
282,2023,1.224647e-16,-1.000000,0.464723,-0.885456
283,2023,1.224647e-16,-1.000000,0.354605,-0.935016
284,2023,1.224647e-16,-1.000000,0.239316,-0.970942
285,2024,5.000000e-01,-0.866025,0.822984,-0.568065


In [27]:
cye_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   g_year       287 non-null    int32  
 1   g_month_sin  287 non-null    float64
 2   g_month_cos  287 non-null    float64
 3   g_week_sin   287 non-null    Float64
 4   g_week_cos   287 non-null    Float64
dtypes: Float64(2), float64(2), int32(1)
memory usage: 10.8 KB


In [28]:
# Reset index for all DataFrames before concatenation - dont do:
#ohe_df.reset_index(drop=True, inplace=True)
#sts_df.reset_index(drop=True, inplace=True)
#cye_df.reset_index(drop=True, inplace=True)

# Combine all features
X = pd.concat([ohe_df, sts_df, cye_df], axis=1).reset_index(drop=True)  # Reset index for consistency
y = df[target_variable].reset_index(drop=True)  # Reset index for consistency

X.shape

(287, 31)

In [29]:
# Final df for model:
df_final = pd.concat([ohe_df, tae_df, sts_df, cye_df, df[target_variable]], axis=1).reset_index(drop=True)  # Reset index for consistency

df_final.shape

(287, 33)

In [30]:
# Final df for validation:
df_final_chk = pd.concat([df, ohe_df, tae_df, sts_df, cye_df], axis=1)

df_final_chk.shape

(287, 62)

In [31]:
# Define all DataFrames to check
dfs_to_check = [ohe_df, tae_df, sts_df, cye_df, X, y, df_final, df_final_chk]

# Perform assertions in a loop
for i, df_check in enumerate(dfs_to_check, 1):
    assert df.shape[0] == df_check.shape[0], f"Row count mismatch in DataFrame {i} ({df_check})!"

print("✅ Row count matches for all DataFrames.")

✅ Row count matches for all DataFrames.


In [32]:
import pandas as pd

# Define all DataFrames to check
dfs_to_check = [ohe_df, tae_df, sts_df, cye_df, X, y, df_final, df_final_chk]

# Perform index equality check in a loop
for i, df_check in enumerate(dfs_to_check, 1):
    pd.testing.assert_index_equal(df.index, df_check.index, 
                                  obj=f"Index mismatch in DataFrame {i} ({df_check})!")

print("✅ Index matches for all DataFrames.")


✅ Index matches for all DataFrames.


In [33]:
df_final

,groda_Höstvete,jordart_Leriga jordar (5-15 % ler),jordart_Lättlera (15-25 % ler),jordart_Unknown,sort,utvecklingsstadium,latitud,longitud,WGS84N,WGS84E,...,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum,g_year,g_month_sin,g_month_cos,g_week_sin,g_week_cos,Bladfläcksvampar
0,1.0,0.0,0.0,1.0,9.480673,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,...,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065,18.0
1,1.0,0.0,0.0,1.0,7.425422,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,...,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065,22.0
2,1.0,0.0,0.0,1.0,9.163897,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,...,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065,20.0
3,1.0,0.0,0.0,1.0,9.655908,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,...,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065,20.0
4,1.0,0.0,0.0,1.0,9.480673,-1.011156,-1.188478,0.558911,-2.842171e-14,3.552714e-15,...,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.748511,-0.663123,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,1.0,0.0,0.0,1.0,10.393924,0.243093,1.557844,-1.765834,-2.842171e-14,3.552714e-15,...,0.224502,-0.086721,-0.173382,-0.658066,2023,1.224647e-16,-1.000000,0.464723,-0.885456,0.0
283,1.0,0.0,0.0,1.0,10.393924,1.183779,1.557844,-1.765834,-2.842171e-14,3.552714e-15,...,0.292568,-0.086721,-0.173382,-0.658066,2023,1.224647e-16,-1.000000,0.354605,-0.935016,0.0
284,1.0,0.0,0.0,1.0,10.393924,1.654122,1.557844,-1.765834,-2.842171e-14,3.552714e-15,...,0.946009,0.440588,0.167704,-0.303556,2023,1.224647e-16,-1.000000,0.239316,-0.970942,0.0
285,1.0,0.0,0.0,1.0,10.393924,-1.011156,1.557844,-1.765834,-2.842171e-14,3.552714e-15,...,-0.946246,-0.824955,-0.779757,-0.658066,2024,5.000000e-01,-0.866025,0.822984,-0.568065,38.0


In [34]:
df_final_chk

,latitud,longitud,groda,sort,jordart,utvecklingsstadium,Bladfläcksvampar,Sädesbladlus,WGS84N,WGS84E,...,G3_7d_sum,G5_7d_sum,G7_7d_sum,G8_7d_sum,G10_7d_sum,g_year,g_month_sin,g_month_cos,g_week_sin,g_week_cos
0,6135000,397000,Höstvete,Elixer,Unknown,31,18.0,NaN,69.042,20.262,...,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065
1,6135000,397000,Höstvete,Mariboss,Unknown,31,22.0,NaN,69.042,20.262,...,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065
2,6135000,397000,Höstvete,Praktik,Unknown,31,20.0,NaN,69.042,20.262,...,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065
3,6135000,397000,Höstvete,Torp,Unknown,31,20.0,NaN,69.042,20.262,...,-1.117355,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.822984,-0.568065
4,6135000,397000,Höstvete,Elixer,Unknown,31,16.0,NaN,69.042,20.262,...,-1.087375,-0.973473,-0.824955,-0.779757,-0.658066,2017,5.000000e-01,-0.866025,0.748511,-0.663123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,6140000,382000,Höstvete,Kask,Unknown,47,0.0,0.08,69.042,20.262,...,0.494031,0.224502,-0.086721,-0.173382,-0.658066,2023,1.224647e-16,-1.000000,0.464723,-0.885456
283,6140000,382000,Höstvete,Kask,Unknown,59,0.0,1.90,69.042,20.262,...,0.651422,0.292568,-0.086721,-0.173382,-0.658066,2023,1.224647e-16,-1.000000,0.354605,-0.935016
284,6140000,382000,Höstvete,Kask,Unknown,65,0.0,1.00,69.042,20.262,...,1.303472,0.946009,0.440588,0.167704,-0.303556,2023,1.224647e-16,-1.000000,0.239316,-0.970942
285,6140000,382000,Höstvete,Kask,Unknown,31,38.0,0.00,69.042,20.262,...,-1.019922,-0.946246,-0.824955,-0.779757,-0.658066,2024,5.000000e-01,-0.866025,0.822984,-0.568065


In [35]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   groda_Höstvete                      287 non-null    float64
 1   jordart_Leriga jordar (5-15 % ler)  287 non-null    float64
 2   jordart_Lättlera (15-25 % ler)      287 non-null    float64
 3   jordart_Unknown                     287 non-null    float64
 4   sort                                287 non-null    float64
 5   utvecklingsstadium                  287 non-null    float64
 6   latitud                             287 non-null    float64
 7   longitud                            287 non-null    float64
 8   WGS84N                              287 non-null    float64
 9   WGS84E                              287 non-null    float64
 10  TM_7d_avg                           287 non-null    float64
 11  TX_7d_avg                           287 non-n

In [36]:
df_final_chk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 62 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   latitud                             287 non-null    object 
 1   longitud                            287 non-null    object 
 2   groda                               287 non-null    object 
 3   sort                                287 non-null    object 
 4   jordart                             287 non-null    object 
 5   utvecklingsstadium                  287 non-null    int64  
 6   Bladfläcksvampar                    287 non-null    float64
 7   Sädesbladlus                        231 non-null    float64
 8   WGS84N                              287 non-null    float64
 9   WGS84E                              287 non-null    float64
 10  TM_7d_avg                           287 non-null    float64
 11  TX_7d_avg                           287 non-n

In [37]:
# Function to clean and rename columns:
def auto_rename_columns(columns):
    renamed_columns = {}
    for col in columns:
        new_col = re.sub(r'\s?\(.*\)', '', col)  # Remove any text in and with parentheses ()
        new_col = new_col.replace(' ', '_')  # Replace spaces with underscores
        renamed_columns[col] = new_col  # Store renamed column
    return renamed_columns
 
# Appy column renames to df
df_final.rename(columns=auto_rename_columns(ohe_df.columns), inplace=True)
df_final_chk.rename(columns=auto_rename_columns(ohe_df.columns), inplace=True)

#X.columns = X.columns.str.replace(' ', '_')

In [38]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   groda_Höstvete         287 non-null    float64
 1   jordart_Leriga_jordar  287 non-null    float64
 2   jordart_Lättlera       287 non-null    float64
 3   jordart_Unknown        287 non-null    float64
 4   sort                   287 non-null    float64
 5   utvecklingsstadium     287 non-null    float64
 6   latitud                287 non-null    float64
 7   longitud               287 non-null    float64
 8   WGS84N                 287 non-null    float64
 9   WGS84E                 287 non-null    float64
 10  TM_7d_avg              287 non-null    float64
 11  TX_7d_avg              287 non-null    float64
 12  TN_7d_avg              287 non-null    float64
 13  UM_7d_avg              287 non-null    float64
 14  UX_7d_avg              287 non-null    float64
 15  UN_7d_

In [39]:
df_final_chk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 62 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   latitud                287 non-null    object 
 1   longitud               287 non-null    object 
 2   groda                  287 non-null    object 
 3   sort                   287 non-null    object 
 4   jordart                287 non-null    object 
 5   utvecklingsstadium     287 non-null    int64  
 6   Bladfläcksvampar       287 non-null    float64
 7   Sädesbladlus           231 non-null    float64
 8   WGS84N                 287 non-null    float64
 9   WGS84E                 287 non-null    float64
 10  TM_7d_avg              287 non-null    float64
 11  TX_7d_avg              287 non-null    float64
 12  TN_7d_avg              287 non-null    float64
 13  UM_7d_avg              287 non-null    float64
 14  UX_7d_avg              287 non-null    float64
 15  UN_7d_

In [40]:
df_final.shape

(287, 33)

In [41]:
df_final_chk.shape

(287, 62)

In [43]:
# pickle - pandas dataframe
df_final.to_pickle(df_5_final_pkl)

# Save the DataFrame to a CSV file with UTF-8 encoding
df_final_chk.to_csv(df_5_final_csv, index=False, encoding='utf-8-sig')  # Ensure proper encoding for special characters

print(f"Data pickled to {df_5_final_pkl} and saved to {df_5_final_csv}")

# Save the DataFrame to a CSV file with UTF-8 encoding
df_final_chk.to_csv(df_5_final_chk_csv, index=False, encoding='utf-8-sig')  # Ensure proper encoding for special characters

print(f"Data saved to {df_5_final_chk_csv}")

Data pickled to df_5_final_t1.pkl and saved to df_5_final_t1.csv
Data saved to df_5_final_chk_t1.csv
